In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

with open('../input/riiid-x-y-nn/X.pkl', 'rb') as f:
    X = pickle.load(f)

with open('../input/riiid-x-y-nn/y.pkl', 'rb') as f:
    y = pickle.load(f)  

with open('../input/riiidxynn12/X1.pkl', 'rb') as f:
    X1 = pickle.load(f)

with open('../input/riiidxynn12/y1.pkl', 'rb') as f:
    y1 = pickle.load(f)  
    
with open('../input/riiidxynn12/X2.pkl', 'rb') as f:
    X2 = pickle.load(f)

with open('../input/riiidxynn12/y2.pkl', 'rb') as f:
    y2 = pickle.load(f)  

print(X.shape)
print(y.shape)
print(X1.shape)
print(y1.shape)
print(X2.shape)
print(y2.shape)

(300000, 564)
(300000,)
(93382, 564)
(93382,)
(274, 564)
(274,)


In [62]:
from sklearn.model_selection import train_test_split
X_big, X_test, y_big, y_test = train_test_split(X, y, test_size=0.1, random_state=40)
X_train, X_valid, y_train, y_valid = train_test_split(X_big, y_big, test_size=0.2, random_state=10)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)
print(X_test.shape, y_test.shape)

(216000, 564) (216000,)
(54000, 564) (54000,)
(30000, 564) (30000,)


In [63]:
import lightgbm as lgb
params = {'objective': 'binary',
          'metric': 'auc',
          'seed': 2020,
          'learning_rate': 0.1, #default
          "boosting_type": "gbdt" #default
         }
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_valid, y_valid)

In [67]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=10000,
    early_stopping_rounds=8
)

Training until validation scores don't improve for 8 rounds
[50]	training's auc: 0.854047	valid_1's auc: 0.848428
[100]	training's auc: 0.907753	valid_1's auc: 0.900658
[150]	training's auc: 0.925362	valid_1's auc: 0.91434
[200]	training's auc: 0.935611	valid_1's auc: 0.920138
[250]	training's auc: 0.943536	valid_1's auc: 0.924482
[300]	training's auc: 0.949959	valid_1's auc: 0.927129
[350]	training's auc: 0.955448	valid_1's auc: 0.929447
[400]	training's auc: 0.959671	valid_1's auc: 0.930841
[450]	training's auc: 0.963484	valid_1's auc: 0.932275
[500]	training's auc: 0.967008	valid_1's auc: 0.933863
[550]	training's auc: 0.970005	valid_1's auc: 0.934866
[600]	training's auc: 0.973022	valid_1's auc: 0.935965
[650]	training's auc: 0.975293	valid_1's auc: 0.93674
[700]	training's auc: 0.977482	valid_1's auc: 0.937562
Early stopping, best iteration is:
[710]	training's auc: 0.977917	valid_1's auc: 0.937728


In [24]:
print(X2.shape)
print(y2.shape)

(274, 564)
(274,)


In [31]:
len(X2)

274

In [33]:
ini_rate = np.ones(188) * 0.25
for i in range(len(X2)):
    X2[i,:188] = ini_rate

In [40]:
X_X2 = np.concatenate((X,X2),axis=0)
y_y2 = np.concatenate((y,y2),axis=0)
print(X_X2.shape, y_y2.shape)

(300274, 564) (300274,)


In [37]:
print(X2[273])

[0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0.25       0.25       0.25       0.25
 0.25       0.25       0

In [65]:
predict_prob = model.predict(X_test)

In [66]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, predict_prob)

0.9383208007830925

In [53]:
print(y2[40:50])
print(predict_prob[40:50])

[1. 1. 0. 1. 1. 0. 1. 0. 1. 0.]
[0.44302056 0.44302056 0.44302056 0.44302056 0.44302056 0.16556951
 0.44302056 0.44302056 0.44302056 0.44302056]


In [ ]:
# def predict_y(row_data, user_ele, user_num, ques_ele=ques_ele_vec, ques_num=ques_num_vec, q_data=questions_data):
#     y = np.ones(len(row_data))*0.25
#     i = 0
#     for index, row in row_data.iterrows():
#         mask = q_data.vec[row.content_id]
#         ques_prob_vec = ques_ele/ques_num
#         if row.user_id in user_ele.keys():
#             borrow_index = (user_num[row.user_id] == 0)
#             user_prob_vec = user_ele[row.user_id]/user_num[row.user_id]
#             user_prob_vec[borrow_index] = ques_prob_vec[borrow_index]
#             y[i] = sum(user_prob_vec * mask)/sum(mask)
#         else:
#             y[i] = sum(ques_prob_vec * mask)/sum(mask)
#         i = i + 1
#     return y

In [ ]:
# import riiideducation
# env = riiideducation.make_env()
# iter_test = env.iter_test()
# for (test_df, sample_prediction_df) in iter_test:
#     test_df = test_df[test_df.content_type_id == 0]
#     test_df['answered_correctly'] = predict_y(test_df,user_ele_dict, user_num_dict)
#     #print('-----------------------------------------')
#     #print(test_df[['row_id', 'answered_correctly']])
#     #print('-----------------------------------------')
#     env.predict(test_df[['row_id', 'answered_correctly']])